# Combine DLC positions with my tracklets to impute missing values

In [38]:
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.projects.utils_project import load_config, safe_cd
from pathlib import Path
import zarr
import os
import pandas as pd
import numpy as np
import napari
from DLC_for_WBFM.utils.visualization.napari_from_config import dlc_to_napari_tracks
from scipy.spatial import distance
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
project_fname = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
project_cfg = load_config(project_fname)
segment_fname = project_cfg['subfolder_configs']['segmentation']
project_dir = Path(project_fname).parent

tracklet_fname = os.path.join('2-training_data', 'all_tracklets.h5')
dlc_fname = os.path.join('3-tracking', 'full_3d_tracks.h5')

with safe_cd(project_dir):
    segment_cfg = load_config(segment_fname)
    df_tracklet = pd.read_hdf(tracklet_fname)
    dlc_tracks = pd.read_hdf(dlc_fname)

In [61]:
dlc_tracks.replace(0, np.NaN, inplace=True)

In [11]:
red_data_fname = project_cfg['preprocessed_red']
red_data = zarr.open(red_data_fname)

In [13]:
napari_tracklets = dlc_to_napari_tracks(df_tracklet, 0.0)

In [16]:
# Visualize everything
# viewer = napari.view_image(red_data)
# viewer.add_tracks(napari_tracklets)

In [124]:
# For one DLC track, look at the distribution of tracklet distances
def calc_dlc_to_tracklet_distances(dlc_tracks, df_tracklet, dlc_name, all_covering_ind):
    coords = ['z', 'x', 'y']

    this_dlc = dlc_tracks[dlc_name][coords]

    all_tracklet_names = list(df_tracklet.columns.levels[0])
    all_dist = []
    for i, name in enumerate(tqdm(all_tracklet_names, leave=False)):
        if i not in all_covering_ind:
            this_tracklet = df_tracklet[name][coords]

            dist = np.linalg.norm(this_tracklet - this_dlc, axis=1)
        else:
            all_dist.append(np.inf)
        
        all_dist.append(dist)
    
    return all_dist

def calc_covering_from_distances(all_dist, df_tracklet, all_covering_ind, d_max=5):
    all_meds = list(map(np.nanmedian, all_dist))
    ind = np.argsort(all_meds)
    all_tracklet_names = list(df_tracklet.columns.levels[0])

#     d_max = 5 # Mean allowed pixel distance
    covering_ind = []
    covering_time_points = []

    t = df_tracklet.index
    for i in ind:
        # Check if this was used before
        for old_coverings in all_covering_ind:
            if i in old_coverings:
                continue
        # Check distance
        if all_meds[i] > d_max:
            break
        # Check time overlap
        name = all_tracklet_names[i]
        is_nan = df_tracklet[name]['x'].isnull()
        new_t = list(t[~is_nan])
        if any([t in covering_time_points for t in new_t]):
            continue

        # Save
        covering_time_points.extend(new_t)
        covering_ind.append(i)
    print(f"Covering of length {len(covering_time_points)} made from {len(covering_ind)} tracklets")
    
    return covering_time_points, covering_ind

In [ ]:
all_dlc_names = list(dlc_tracks.columns.levels[0])
dlc_name = all_dlc_names[0]

all_all_dist = []
all_covering_time_points = []
all_covering_ind = []
for dlc_name in tqdm(all_dlc_names):
    dist = calc_dlc_to_tracklet_distances(dlc_tracks, df_tracklet, dlc_name, all_covering_ind)
    covering_time_points, covering_ind = calc_covering_from_distances(dist, df_tracklet, all_covering_ind, d_max=5)
    
    all_all_dist.append(dist)
    all_covering_time_points.append(covering_time_points)
    all_covering_ind.append(covering_ind)

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 770 made from 8 tracklets


  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 309 made from 5 tracklets


...

  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 769 made from 13 tracklets


  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 325 made from 10 tracklets


  0%|          | 0/5550 [00:00<?, ?it/s]

In [119]:
%matplotlib widget
plt.plot(covering_time_points, np.ones_like(covering_time_points), 'o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [104]:
# %matplotlib widget
# # # from ipywidgets import interact
# best_tracklets = np.argpartition(all_means, 10)[:10]
# print(best_tracklets)
# plt.hist([all_means[i] for i in best_tracklets])

In [64]:
d = all_dist[0]
np.round(d[~np.isnan(d)], 1)

array([1.1, 0.8, 3.2, 2. , 1.2, 2.4, 0.6, 2.8, 1.7, 1.2, 0.8, 0.9, 2.9,
       0.6, 2.2, 3.7, 0.6, 0.6, 1.6, 2.2])

In [67]:
name = 'neuron1'
this_tracklet = df_tracklet[name][coords]
this_tracklet.iloc[1:20, :]

coords,z,x,y
1,8.444737,484.360526,289.947368
2,6.431944,480.381944,283.933333
3,8.410738,472.357047,267.971812
4,8.450135,486.397574,247.900270
5,8.447958,490.371542,235.891963
6,10.436702,480.283391,217.919861
7,8.451697,452.390339,199.921671
8,8.459060,446.424161,187.893960
9,8.443312,432.313376,177.970701
10,8.426020,406.274235,177.982143


In [68]:
this_dlc.iloc[1:20, :]

coords,z,x,y
1,9,484.520248,290.520874
2,9,482.337982,284.265335
3,9,473.891083,269.164436
4,9,NaN,NaN
5,9,491.375468,235.572243
6,9,482.158783,218.350449
7,9,452.336416,199.883942
8,9,449.116119,187.569519
9,9,430.832611,178.534673
10,9,405.493876,178.703758


In [73]:
len(best_tracklets)

5550